In [34]:
%matplotlib inline

In [35]:
from os import path
import pandas as pd
from skimage.data import imread
from skimage.io import imshow
import numpy as np
import pandas as pd
from numpy.random import randint, seed

In [36]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD

In [37]:
nb_img = 256
nb_img_depth = 3

In [38]:
nb_pool = 2
nb_conv = 3
nb_filters = 32

In [39]:
model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv, border_mode='valid', 
                        input_shape=(nb_img_depth, nb_img, nb_img)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))

#model.add(Convolution2D(nb_filters, nb_conv, nb_conv, border_mode='valid'))
#model.add(Activation('relu'))
#model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='mse', optimizer='rmsprop')

In [40]:
from transform import load_training, transform_img

In [41]:
training = load_training()

In [42]:
def transform_pcf(training,
                  output_dir="data-%d-patches" % nb_img,
                  patch_size=nb_img):
    patch_num = 0
    num_per_img_dict = {}
    for i, row in training.iterrows():
        row_patches = []
        row_output = []
        name = row['name']
        print("Transforming image %s" % name)
        import sys
        sys.stdout.flush()
        for kind in ["DX", "TS"]:
            num_per_img = 0
            img = imread("images/%s/%s-%s.png" % (kind, name, kind))
            img_patches = transform_img(img, name, patch_size=patch_size)
            from os import mkdir, path
            if not path.exists(output_dir):
                mkdir(output_dir)
            for img_patch in img_patches:
                with open(path.join(output_dir, "patch_file_%s_%d" % (
                          name, patch_num)), 'w') as f:
                    np.save(f, img_patch)
                patch_num += 1
                num_per_img += 1
            num_per_img_dict[(name, kind)] = num_per_img
    return num_per_img_dict

In [43]:
# num = transform_pcf(training)

In [45]:
"""
df_patches = pd.DataFrame({'name': [k[0] for k in num.keys()],
              'kind': [k[1] for k in num.keys()],
              'patches': num.values()})
"""

"\ndf_patches = pd.DataFrame({'name': [k[0] for k in num.keys()],\n              'kind': [k[1] for k in num.keys()],\n              'patches': num.values()})\n"

In [47]:
# df_patches.to_csv('df_patches.csv')

In [48]:
fga_dict = training.set_index('name').to_dict()['fga']

In [49]:
def get_x_y(training, output_dir="data-%d-patches" % nb_img):
    from os import listdir
    from numpy.random import choice, seed
    seed(1234)
    patches = []
    outputs = []
    patch_files = choice(listdir(output_dir), size=5000, replace=False)
    for patch_file in patch_files:
        name = patch_file.split("_")[2]
        patch = np.load(path.join(output_dir, 
                                  patch_file))
        from skimage.feature import canny
        from skimage.color import rgb2gray
        #for rot in range(4):
        #patches.append(np.rot90(patch, k=rot))
        patches.append(patch)
        outputs.append(fga_dict[name])
    print("Merging X...")
    import sys
    sys.stdout.flush()
    X = np.asanyarray(patches)
    print("Merging y...")
    sys.stdout.flush()
    y = np.asanyarray(outputs)
    return X, y

In [50]:
X_train, y_train = get_x_y(training)

Merging X...


KeyboardInterrupt: 

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.transpose((0, 3, 2, 1))

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [28]:
model.fit(X_train, y_train, batch_size=32, nb_epoch=10)

Epoch 1/10
5000/5000 [==============================] - 40s - loss: 0.0181    
Epoch 2/10
5000/5000 [==============================] - 40s - loss: 0.0178    
Epoch 3/10
5000/5000 [==============================] - 40s - loss: 0.0179    
Epoch 4/10
5000/5000 [==============================] - 40s - loss: 0.0178    
Epoch 5/10
5000/5000 [==============================] - 40s - loss: 0.0178    
Epoch 6/10
5000/5000 [==============================] - 40s - loss: 0.0179    
Epoch 7/10
5000/5000 [==============================] - 40s - loss: 0.0178    
Epoch 8/10
5000/5000 [==============================] - 40s - loss: 0.0178    
Epoch 9/10
5000/5000 [==============================] - 40s - loss: 0.0179    
Epoch 10/10
5000/5000 [==============================] - 40s - loss: 0.0178    


In [ ]:
def get_x_y_pair(training, output_dir="data-%d-patches" % nb_img):
    from os import listdir
    from numpy.random import choice, seed
    seed(1234)
    patches = []
    outputs = []
    patch_files = choice(listdir(output_dir), size=2, replace=False)
    for patch_file in patch_files:
        name = patch_file.split("_")[2]
        patch = np.load(path.join(output_dir, 
                                  patch_file))
        for i in range(1000):
            patches.append(patch)
            outputs.append(fga_dict[name])
    print("Merging X...")
    import sys
    sys.stdout.flush()
    X = np.asanyarray(patches)
    print("Merging y...")
    sys.stdout.flush()
    y = np.asanyarray(outputs)
    return X, y

In [ ]:
X_single, y_single = get_x_y_single(training)

In [ ]:
X_single = X_single.transpose((0, 3, 2, 1))

In [ ]:
y_single

In [ ]:
model.fit(X_single, y_single, batch_size=32, nb_epoch=10)